In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.linalg
import matplotlib.pyplot as plt
import linear_shrinkage
import Util
from os.path import join
from generating_MTP import generate_mat as generate_MTP2
import os

In [3]:
def generate_vols(old_r, old_d, w, a, b):
    return np.sqrt(w + a*old_r**2 + b*old_d**2)

In [20]:
N = 5
T = 1250

student_t = False

#C = np.eye(N) #uncond_corr_mat
C = Util.cov_to_corr(np.linalg.inv(generate_MTP2(N, 25, sparse_thres = 0.3)))

C_inv = generate_MTP2(N, 25, sparse_thres = 0.3)
print("generated C!")
alpha = 0.05
beta = 0.93
w = 0
a = np.ones(N) * 0.05
b = np.ones(N) * 0.9

r_0 = np.zeros(N)
d_0 = np.ones(N)
Q_0 = C
r_series = [r_0] #first return is 0
d_series = [d_0]
Q_series = [Q_0]

Q_inv_series = [C_inv]

H_series = [None]

for t in range(T + 500):
    old_r = r_series[-1] #both vectors of length N
    old_d = d_series[-1]
    s_t_1 = (old_r / old_d).reshape(-1, 1)
    Q_t = (1-alpha-beta)*C + alpha*(s_t_1.dot(s_t_1.T)) + beta*Q_series[-1]
    
    #print(s_t_1.shape)
    #Q_t_inv = (1-alpha-beta)*C_inv + alpha*np.linalg.inv(s_t_1.dot(s_t_1.T)) + beta*Q_inv_series[-1]
    Q_t_inv = None
    #Q_t = np.linalg.inv(Q_t_inv)
    
    #conditional variances at the t-th time step
    d_t = generate_vols(old_r, old_d, w, a, b)
    D_t = np.diag(d_t) 
    
    #Q_t is the conditional covariance matrix, of the devolatized returns
    #R_t is the conditional correlation matrix of the devolatized returns
    #H_t is the conditional covariance matrix of the returns (after factoring in the vols)
    
    Q_t_diag = np.diag(np.power(np.diag(Q_t), -0.5))
    R_t = Q_t_diag.dot(Q_t).dot(Q_t_diag)
    H_t = D_t.dot(R_t).dot(D_t)
    
    r_t = np.random.multivariate_normal(mean = np.zeros(N),
                                        cov = H_t)
    
    if student_t:
        r_t = np.random.standard_t(5, size = N)
        r_t = scipy.linalg.sqrtm(H_t).dot(r_t)
    
    r_series.append(r_t)
    d_series.append(d_t)
    Q_series.append(Q_t)
    Q_inv_series.append(Q_t_inv)
    H_series.append(H_t)

KeyboardInterrupt: 

In [19]:
for r in r_series:
    rs = r.reshape(-1,1)
    print(rs.dot(rs.T))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[ 5.38782897e+00  1.36956183e+00 -1.38471709e+00 ... -3.92395042e-01
   9.82603287e-02 -2.38587135e-01]
 [ 1.36956183e+00  3.48136442e-01 -3.51988840e-01 ... -9.97450504e-02
   2.49773325e-02 -6.06477739e-02]
 [-1.38471709e+00 -3.51988840e-01  3.55883869e-01 ...  1.00848806e-01
  -2.52537260e-02  6.13188883e-02]
 ...
 [-3.92395042e-01 -9.97450504e-02  1.00848806e-01 ...  2.85780915e-02
  -7.15628987e-03  1.73762771e-02]
 [ 9.82603287e-02  2.49773325e-02 -2.52537260e-02 ... -7.15628987e-03
   1.79201906e-03 -4.35122392e-03]
 [-2.38587135e-01 -6.06477739e-02  6.13188883e-02 ...  1.73762771e-02
  -4.35122392e-03  1.05652613e-02]]
[[ 3.95143465e+00  2.57142877e+00  1.02327932e+00 ...  2.46376879e-02
   1.70971668e-01 -1.94106445e+00]
 [ 2.57142877e+00  1.67337854e+00  6.65907477e-01 ...  1.60331792e-02
   1.11261227e-01 -1.26316374e+00

  -1.04584668e-04  2.21237826e-04]]
[[ 4.61463153e-05  2.06800114e-05 -1.51121444e-06 ...  1.16991438e-04
   2.08396739e-04 -8.92516516e-05]
 [ 2.06800114e-05  9.26754106e-06 -6.77235694e-07 ...  5.24285472e-05
   9.33909223e-05 -3.99972384e-05]
 [-1.51121444e-06 -6.77235694e-07  4.94897386e-08 ... -3.83127340e-06
  -6.82464373e-06  2.92284191e-06]
 ...
 [ 1.16991438e-04  5.24285472e-05 -3.83127340e-06 ...  2.96599987e-04
   5.28333281e-04 -2.26273300e-04]
 [ 2.08396739e-04  9.33909223e-05 -6.82464373e-06 ...  5.28333281e-04
   9.41119581e-04 -4.03060418e-04]
 [-8.92516516e-05 -3.99972384e-05  2.92284191e-06 ... -2.26273300e-04
  -4.03060418e-04  1.72621741e-04]]
[[ 2.26407522e-06  1.17632424e-05  7.14503175e-06 ... -2.08533540e-05
   1.36048888e-05 -3.67068344e-05]
 [ 1.17632424e-05  6.11171708e-05  3.71227686e-05 ... -1.08345808e-04
   7.06856396e-05 -1.90714243e-04]
 [ 7.14503175e-06  3.71227686e-05  2.25484906e-05 ... -6.58095964e-05
   4.29346877e-05 -1.15840452e-04]
 ...
 [-2.085

[[ 6.61389047e-08 -1.88395939e-08  1.68667385e-08 ... -5.32223543e-08
   7.94450453e-08 -1.02145421e-08]
 [-1.88395939e-08  5.36643751e-09 -4.80447183e-09 ...  1.51603288e-08
  -2.26298334e-08  2.90960104e-09]
 [ 1.68667385e-08 -4.80447183e-09  4.30135439e-09 ... -1.35727608e-08
   2.02600695e-08 -2.60491177e-09]
 ...
 [-5.32223543e-08  1.51603288e-08 -1.35727608e-08 ...  4.28283324e-08
  -6.39298816e-08  8.21970038e-09]
 [ 7.94450453e-08 -2.26298334e-08  2.02600695e-08 ... -6.39298816e-08
   9.54281787e-08 -1.22695525e-08]
 [-1.02145421e-08  2.90960104e-09 -2.60491177e-09 ...  8.21970038e-09
  -1.22695525e-08  1.57754156e-09]]
[[ 1.21848769e-07 -1.38731832e-08 -9.08157344e-08 ...  2.25474848e-08
  -1.55713667e-07  3.87779793e-08]
 [-1.38731832e-08  1.57954169e-09  1.03398937e-08 ... -2.56716081e-09
   1.77288966e-08 -4.41509596e-09]
 [-9.08157344e-08  1.03398937e-08  6.76863431e-08 ... -1.68049821e-08
   1.16055756e-07 -2.89018156e-08]
 ...
 [ 2.25474848e-08 -2.56716081e-09 -1.6804982

[[ 1.12360673e-14  3.11535959e-12  1.01893035e-12 ... -1.23909976e-13
   4.63842309e-13  3.44514959e-13]
 [ 3.11535959e-12  8.63777793e-10  2.82512943e-10 ... -3.43558046e-11
   1.28606883e-10  9.55216765e-11]
 [ 1.01893035e-12  2.82512943e-10  9.24005729e-11 ... -1.12366393e-11
   4.20630274e-11  3.12419585e-11]
 ...
 [-1.23909976e-13 -3.43558046e-11 -1.12366393e-11 ...  1.36646406e-12
  -5.11519626e-12 -3.79926884e-12]
 [ 4.63842309e-13  1.28606883e-10  4.20630274e-11 ... -5.11519626e-12
   1.91481310e-11  1.42221126e-11]
 [ 3.44514959e-13  9.55216765e-11  3.12419585e-11 ... -3.79926884e-12
   1.42221126e-11  1.05633541e-11]]
[[ 6.78827028e-13 -1.88076794e-11 -8.94313799e-12 ... -5.44805550e-12
   3.48308564e-12 -1.97937617e-12]
 [-1.88076794e-11  5.21088273e-10  2.47779869e-10 ...  1.50944610e-10
  -9.65028723e-11  5.48408811e-11]
 [-8.94313799e-12  2.47779869e-10  1.17820467e-10 ...  7.17748560e-11
  -4.58875593e-11  2.60770910e-11]
 ...
 [-5.44805550e-12  1.50944610e-10  7.1774856

  -5.75646788e-14  1.79516707e-13]]
[[ 7.12879667e-15  6.52957347e-14  1.66993012e-15 ... -1.78185437e-15
   1.36491060e-14  9.25185515e-15]
 [ 6.52957347e-14  5.98071900e-13  1.52956129e-14 ... -1.63207755e-14
   1.25018071e-13  8.47417464e-14]
 [ 1.66993012e-15  1.52956129e-14  3.91183357e-16 ... -4.17401762e-16
   3.19732128e-15  2.16725940e-15]
 ...
 [-1.78185437e-15 -1.63207755e-14 -4.17401762e-16 ...  4.45377411e-16
  -3.41161635e-15 -2.31251631e-15]
 [ 1.36491060e-14  1.25018071e-13  3.19732128e-15 ... -3.41161635e-15
   2.61331756e-14  1.77140067e-14]
 [ 9.25185515e-15  8.47417464e-14  2.16725940e-15 ... -2.31251631e-15
   1.77140067e-14  1.20071911e-14]]
[[2.04174849e-14 8.53367838e-14 1.30074085e-14 ... 1.52544628e-14
  3.35983422e-14 5.50983512e-14]
 [8.53367838e-14 3.56673052e-13 5.43656778e-14 ... 6.37574508e-14
  1.40427407e-13 2.30288701e-13]
 [1.30074085e-14 5.43656778e-14 8.28665608e-15 ... 9.71819150e-15
  2.14045639e-14 3.51016183e-14]
 ...
 [1.52544628e-14 6.3757450

...

[[ 2.47459340e-28  2.52460830e-27 -1.53675072e-28 ... -8.03216211e-29
   6.53750468e-28 -1.50756348e-27]
 [ 2.52460830e-27  2.57563407e-26 -1.56781054e-27 ... -8.19450303e-28
   6.66963654e-27 -1.53803339e-26]
 [-1.53675072e-28 -1.56781054e-27  9.54339724e-29 ...  4.98806427e-29
  -4.05986496e-28  9.36214114e-28]
 ...
 [-8.03216211e-29 -8.19450303e-28  4.98806427e-29 ...  2.60712035e-29
  -2.12197678e-28  4.89332683e-28]
 [ 6.53750468e-28  6.66963654e-27 -4.05986496e-28 ... -2.12197678e-28
   1.72711070e-27 -3.98275665e-27]
 [-1.50756348e-27 -1.53803339e-26  9.36214114e-28 ...  4.89332683e-28
  -3.98275665e-27  9.18432761e-27]]
[[ 1.94317932e-28  4.08608272e-27  2.47850772e-28 ...  1.33470204e-28
   1.10331469e-27 -2.35273810e-27]
 [ 4.08608272e-27  8.59214167e-26  5.21176171e-27 ...  2.80658758e-27
   2.32003040e-26 -4.94729560e-26]
 [ 2.47850772e-28  5.21176171e-27  3.16131428e-28 ...  1.70240043e-28
   1.40726795e-27 -3.00089626e-27]
 ...
 [ 1.33470204e-28  2.80658758e-27  1.7024004

[[ 4.94847711e-32 -3.45261701e-31 -1.31215270e-31 ... -6.18246818e-32
  -2.45310949e-31  2.49723663e-32]
 [-3.45261701e-31  2.40893590e-30  9.15506046e-31 ...  4.31358867e-31
   1.71156648e-30 -1.74235456e-31]
 [-1.31215270e-31  9.15506046e-31  3.47934255e-31 ...  1.63936139e-31
   6.50473704e-31 -6.62174586e-32]
 ...
 [-6.18246818e-32  4.31358867e-31  1.63936139e-31 ...  7.72417695e-32
   3.06483612e-31 -3.11996715e-32]
 [-2.45310949e-31  1.71156648e-30  6.50473704e-31 ...  3.06483612e-31
   1.21608043e-30 -1.23795559e-31]
 [ 2.49723663e-32 -1.74235456e-31 -6.62174586e-32 ... -3.11996715e-32
  -1.23795559e-31  1.26022424e-32]]
[[ 2.38477103e-31  7.22109154e-31 -3.84452378e-31 ... -2.73328421e-32
   9.28343997e-31 -7.34844313e-31]
 [ 7.22109154e-31  2.18654799e-30 -1.16412259e-30 ... -8.27639016e-32
   2.81102752e-30 -2.22511008e-30]
 [-3.84452378e-31 -1.16412259e-30  6.19781225e-31 ...  4.40636691e-32
  -1.49659675e-30  1.18465312e-30]
 ...
 [-2.73328421e-32 -8.27639016e-32  4.4063669

[[ 6.21401066e-35  2.93743581e-33  1.72697823e-34 ...  2.23229261e-35
   7.14905164e-34 -1.72058877e-33]
 [ 2.93743581e-33  1.38856040e-31  8.16362891e-33 ...  1.05523093e-33
   3.37944066e-32 -8.13342519e-32]
 [ 1.72697823e-34  8.16362891e-33  4.79956341e-34 ...  6.20391717e-35
   1.98684187e-33 -4.78180602e-33]
 ...
 [ 2.23229261e-35  1.05523093e-33  6.20391717e-35 ...  8.01918529e-36
   2.56819243e-34 -6.18096397e-34]
 [ 7.14905164e-34  3.37944066e-32  1.98684187e-33 ...  2.56819243e-34
   8.22479106e-33 -1.97949097e-32]
 [-1.72058877e-33 -8.13342519e-32 -4.78180602e-33 ... -6.18096397e-34
  -1.97949097e-32  4.76411434e-32]]
[[1.13802766e-35 5.02548759e-35 1.08626614e-34 ... 5.61527234e-35
  2.43977663e-34 1.30459975e-33]
 [5.02548759e-35 2.21923653e-34 4.79691066e-34 ... 2.47968327e-34
  1.07739624e-33 5.76106372e-33]
 [1.08626614e-34 4.79691066e-34 1.03685892e-33 ... 5.35986991e-34
  2.32880697e-33 1.24526194e-32]
 ...
 [5.61527234e-35 2.47968327e-34 5.35986991e-34 ... 2.77069570e

[[3.70097452e-38 1.13696221e-36 1.04310907e-37 ... 2.91678283e-38
  1.39960398e-37 1.63915429e-36]
 [1.13696221e-36 3.49281810e-35 3.20449546e-36 ... 8.96053687e-37
  4.29966981e-36 5.03558314e-35]
 [1.04310907e-37 3.20449546e-36 2.93997307e-37 ... 8.22086892e-38
  3.94474376e-37 4.61990945e-36]
 ...
 [2.91678283e-38 8.96053687e-37 8.22086892e-38 ... 2.29875186e-38
  1.10304484e-37 1.29183734e-36]
 [1.39960398e-37 4.29966981e-36 3.94474376e-37 ... 1.10304484e-37
  5.29290675e-37 6.19881832e-36]
 [1.63915429e-36 5.03558314e-35 4.61990945e-36 ... 1.29183734e-36
  6.19881832e-36 7.25978190e-35]]
[[ 3.94103925e-37 -2.90176523e-36 -9.59694909e-37 ...  4.80074619e-37
   1.01268501e-37 -4.35190170e-37]
 [-2.90176523e-36  2.13655356e-35  7.06618011e-36 ... -3.53476265e-36
  -7.45634330e-37  3.20428096e-36]
 [-9.59694909e-37  7.06618011e-36  2.33698337e-36 ... -1.16904486e-36
  -2.46602124e-37  1.05974532e-36]
 ...
 [ 4.80074619e-37 -3.53476265e-36 -1.16904486e-36 ...  5.84799148e-37
   1.23359

[[ 7.53813123e-42  2.54993001e-39 -5.24793256e-41 ...  8.83661248e-41
  -1.28227314e-41 -9.43361756e-41]
 [ 2.54993001e-39  8.62566975e-37 -1.77522257e-38 ...  2.98916836e-38
  -4.33755618e-39 -3.19111777e-38]
 [-5.24793256e-41 -1.77522257e-38  3.65353101e-40 ... -6.15191550e-40
   8.92699103e-41  6.56754138e-40]
 ...
 [ 8.83661248e-41  2.98916836e-38 -6.15191550e-40 ...  1.03587637e-39
  -1.50315118e-40 -1.10586059e-39]
 [-1.28227314e-41 -4.33755618e-39  8.92699103e-41 ... -1.50315118e-40
   2.18120959e-41  1.60470468e-40]
 [-9.43361756e-41 -3.19111777e-38  6.56754138e-40 ... -1.10586059e-39
   1.60470468e-40  1.18057298e-39]]
[[ 2.62107644e-40  2.11485764e-38 -7.10807714e-41 ...  4.29113678e-41
   3.28344517e-40 -2.03964480e-40]
 [ 2.11485764e-38  1.70640686e-36 -5.73526623e-39 ...  3.46237265e-39
   2.64930049e-38 -1.64572017e-38]
 [-7.10807714e-41 -5.73526623e-39  1.92763400e-41 ... -1.16371010e-41
  -8.90434982e-41  5.53129712e-41]
 ...
 [ 4.29113678e-41  3.46237265e-39 -1.1637101

In [17]:
Util.check_positive(H_series[4])

0.06868686868686869

In [114]:
returns = np.array(r_series[501:])

In [115]:
true_covs = np.array(H_series[501:])

In [116]:
folder = "DCC_model_try_2"

In [117]:
if not os.path.isdir('matlab/data/{}'.format(folder)):
    os.mkdir('matlab/data/{}'.format(folder))

In [118]:
np.save(join('matlab/data', folder, 'returns.npy'), returns)
np.save(join('matlab/data', folder, 'true_covs.npy'), true_covs)

In [125]:
lookback = 50
losses = []

uids = []
pastRets = []

for t in range(lookback, T):
    past_ret = returns[t-lookback: t]
    uids.append(t)
    pastRets.append(past_ret)
    est_cov = linear_shrinkage.estimator(past_ret)
    #est_cov = linear_shrinkage.shrink_iden(past_ret, 0.0000005)
    #est_cov = Util.identity_cov(past_ret)
    true_cov = H_series[t]
    loss = Util.loss(est_cov, true_cov)
    losses.append(loss)

In [126]:
print(np.mean(losses))

5.077247601698601e-08


In [127]:
Util.populate_covs_from_samples(folder, uids, pastRets)

Done populating all covs


In [128]:
Util.run_parallel_MTP2(8, folder, uids)

Done with running all commands!


In [129]:
MTP2_losses = []
for t in range(lookback, T):
    est_cov = Util.get_MTP2_cov(folder, uid = t)
    true_cov = H_series[t]
    loss = Util.loss(est_cov, true_cov)
    MTP2_losses.append(loss)

In [130]:
print(np.mean(MTP2_losses))

9.431420157249207e-06


In [131]:
iden_losses = []

for t in range(lookback, T):
    past_ret = returns[t-lookback: t]
    est_cov = Util.identity_cov(past_ret)
    true_cov = H_series[t]
    loss = Util.loss(est_cov, true_cov)
    iden_losses.append(loss)

In [132]:
print(np.mean(iden_losses))

9.308295278669847e-06
